In [2]:
import pandas as pd
import numpy as np

In [3]:
import spacy
from spacy import displacy
from spacy.symbols import amod
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

In [4]:
nyanglerbass_masked_df = pd.read_csv('/home/desbrium/Metis/FishingSpots/Data/nyanglerbass_masked.csv')

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
stop_string = '''
FISHING,FISH,JUST,KNOW,DON,LIKE,TIME,YEAR,THINK,GUYS,NEW,PEOPLE,LL,YEARS,ST,USE,FISHED,FISHING,COLOR,COM,HTTP,PHOTOBUCKET,WWW,JPG,
ALBUMS,SRC,IMG,HREF,_BLANK,BORDER,ALT,VIDEO,TARGET,PHP,MARTY,GUYS,LOL,DAY,TODAY,JOB,GUY,CONGRAT,LAUGH,LOOK,PIC,REPORT,
THANK,WAY,GREJOB,JOHN,MIKE,HOUR,MAN,JIM
'''

In [7]:
stop_list = stop_string.replace('\n', '').lower().split(',')

nlp.Defaults.stop_words |= {stop_word for stop_word in stop_list}

In [7]:
lake_george_ex = [sent for sent in nlp(nyanglerbass_masked_df.iat[6733,5]).sents]
displacy.render(lake_george_ex[0], style='dep', options={'distance': 100}) 

In [8]:
for token in lake_george_ex[0]:
    if token.dep_ == 'dobj':
        print(f"direct object: {token.text} --> NOUN: {token.head}")
for ent in lake_george_ex[0].ents:
    print(ent.text, ent.label_)

direct object: george --> NOUN: lake
the last week DATE


In [9]:
lake_george_ex2 = [sent for sent in nlp(nyanglerbass_masked_df.iat[15529,5]).sents]
displacy.render(lake_george_ex2[0], style='dep', options={'distance': 100}) 

In [10]:
for token in lake_george_ex2[0]:
    if token.dep_ == 'dobj':
        print(f"direct object: {token.text} --> NOUN: {token.head}")
for ent in lake_george_ex2[0].ents:
    print(ent.text, ent.label_)

direct object: trip --> NOUN: doing
annual DATE
a couple of weeks DATE


In [12]:
nyanglerbass_masked_df['Thread_Name'].value_counts()

Places to Fish/Fishing Reports              17215
Tackle and Fishing Tactics                   8706
Multi-Species                                1024
Saltwater                                     685
Fishing Report                                506
General Discussion                            363
Inshore Tackle and Techniques with Lep        172
New York State DEC Updates                     70
Sweetwater - Lakes, Reservoirs and Ponds       57
Long Island Sound Fishing                      33
Name: Thread_Name, dtype: int64

In [8]:
fishing_report = nyanglerbass_masked_df[(nyanglerbass_masked_df['Thread_Name'] == 'Places to Fish/Fishing Reports') | (nyanglerbass_masked_df['Thread_Name'] == 'Fishing Report')]

In [9]:
fishing_tactics = nyanglerbass_masked_df[(nyanglerbass_masked_df['Thread_Name'] == 'Tackle and Fishing Tactics')]

In [10]:
salt_water = nyanglerbass_masked_df[(nyanglerbass_masked_df['Thread_Name'] == 'Saltwater') | (nyanglerbass_masked_df['Thread_Name'] == 'Long Island Sound Fishing')]

In [11]:
species = nyanglerbass_masked_df[(nyanglerbass_masked_df['Thread_Name'] == 'Multi-Species')]

In [29]:
fishing_report['Docs'] = list(nlp.pipe(fishing_report['Lemma_Post_Content']))

<ipython-input-29-771f494d30cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fishing_report['Docs'] = list(nlp.pipe(fishing_report['Lemma_Post_Content']))


In [30]:
fishing_report_nouns = [token.text.lower() for doc in fishing_report['Docs'] for token in doc if token.pos_=='NOUN' or token.dep_=='dobj']
fishing_report_adjs = [token.text.lower() for doc in fishing_report['Docs'] for token in doc if token.pos_=='ADJ']

In [18]:
fishing_tactics['Docs'] = list(nlp.pipe(fishing_tactics['Lemma_Post_Content']))

<ipython-input-18-0b970423d446>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fishing_tactics['Docs'] = list(nlp.pipe(fishing_tactics['Lemma_Post_Content']))


In [19]:
fishing_tactics_nouns = [token.text.lower() for doc in fishing_tactics['Docs'] for token in doc if token.pos_=='NOUN' or token.dep_=='dobj']
fishing_tactics_adjs = [token.text.lower() for doc in fishing_tactics['Docs'] for token in doc if token.pos_=='ADJ']

In [ ]:
salt_water['Docs'] = list(nlp.pipe(salt_water['Lemma_Post_Content']))

In [ ]:
salt_water_nouns = [token.text.lower() for doc in salt_water['Docs'] for token in doc if token.pos_=='NOUN' or token.dep_=='dobj']
salt_water_adjs = [token.text.lower() for doc in salt_water['Docs'] for token in doc if token.pos_=='ADJ']

In [12]:
species['Docs'] = list(nlp.pipe(species['Lemma_Post_Content']))

<ipython-input-12-ea2c4736cf77>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  species['Docs'] = list(nlp.pipe(species['Lemma_Post_Content']))


In [13]:
species_nouns = [token.text.lower() for doc in species['Docs'] for token in doc if token.pos_=='NOUN' or token.dep_=='dobj']
species_adjs = [token.text.lower() for doc in species['Docs'] for token in doc if token.pos_=='ADJ']

In [15]:
from collections import Counter

In [35]:
Counter(fishing_report_nouns).most_common(30)

[('fish', 83904),
 ('water', 40939),
 ('lake', 37143),
 ('day', 36862),
 ('bass', 30211),
 ('time', 28706),
 ('fishing', 28702),
 ('it', 28100),
 ('year', 24884),
 ('guy', 22046),
 ('way', 14069),
 ('boat', 11979),
 ('catch', 11499),
 ('bait', 11362),
 ('one', 10780),
 ('week', 10772),
 ('today', 10408),
 ('area', 10392),
 ('wind', 10326),
 ('report', 10250),
 ('lot', 10082),
 ('jig', 9887),
 ('they', 9681),
 ('weed', 9593),
 ('hour', 9421),
 ('thing', 9336),
 ('spot', 8966),
 ('bite', 8747),
 ('job', 8683),
 ('i', 8609)]

In [34]:
Counter(fishing_report_adjs).most_common(20)

[('nice', 39755),
 ('good', 39406),
 ('big', 20817),
 ('few', 18873),
 ('last', 12743),
 ('other', 12727),
 ('more', 11607),
 ('small', 10445),
 ('first', 10067),
 ('little', 10058),
 ('new', 9456),
 ('many', 8907),
 ('sure', 7859),
 ('next', 7644),
 ('deep', 7378),
 ('same', 7104),
 ('most', 6940),
 ('long', 5988),
 ('bad', 5851),
 ('much', 5047)]

In [16]:
Counter(species_nouns).most_common(30)

[('fish', 5246),
 ('water', 1951),
 ('it', 1908),
 ('time', 1568),
 ('day', 1511),
 ('year', 1498),
 ('fishing', 1468),
 ('bass', 1165),
 ('guy', 1160),
 ('lake', 1098),
 ('bait', 983),
 ('rod', 938),
 ('they', 926),
 ('river', 909),
 ('one', 843),
 ('line', 835),
 ('catch', 835),
 ('trout', 810),
 ('hook', 744),
 ('way', 717),
 ('lot', 627),
 ('thing', 611),
 ('week', 566),
 ('trip', 525),
 ('i', 503),
 ('perch', 474),
 ('ice', 464),
 ('hour', 459),
 ('today', 443),
 ('couple', 430)]

In [17]:
Counter(species_adjs).most_common(30)

[('good', 1993),
 ('nice', 1649),
 ('big', 1453),
 ('few', 1086),
 ('other', 760),
 ('last', 746),
 ('small', 721),
 ('more', 691),
 ('little', 627),
 ('first', 572),
 ('many', 521),
 ('sure', 464),
 ('same', 437),
 ('most', 415),
 ('new', 414),
 ('next', 407),
 ('long', 372),
 ('deep', 323),
 ('large', 319),
 ('lb', 314),
 ('cold', 303),
 ('much', 296),
 ('awesome', 287),
 ('pike', 287),
 ('brown', 268),
 ('bad', 257),
 ('white', 247),
 ('old', 246),
 ('rich', 244),
 ('low', 239)]

In [20]:
Counter(fishing_tactics_nouns).most_common(30)

[('fish', 36514),
 ('it', 29783),
 ('rod', 28146),
 ('bait', 22893),
 ('water', 20453),
 ('line', 19872),
 ('time', 18377),
 ('bass', 17226),
 ('year', 16899),
 ('hook', 15665),
 ('reel', 15335),
 ('fishing', 15051),
 ('they', 14421),
 ('jig', 13669),
 ('guy', 10798),
 ('day', 10032),
 ('one', 9198),
 ('thing', 9052),
 ('way', 8767),
 ('use', 7976),
 ('lake', 7956),
 ('color', 7329),
 ('lot', 7073),
 ('weight', 6225),
 ('rig', 6191),
 ('worm', 5936),
 ('braid', 5813),
 ('i', 5517),
 ('weed', 5211),
 ('action', 5143)]

In [21]:
Counter(fishing_tactics_adjs).most_common(30)

[('good', 24461),
 ('other', 12941),
 ('more', 11743),
 ('big', 11004),
 ('few', 9556),
 ('new', 9118),
 ('many', 8416),
 ('same', 7874),
 ('small', 7331),
 ('last', 6677),
 ('little', 6417),
 ('most', 6180),
 ('different', 6022),
 ('heavy', 5931),
 ('sure', 5638),
 ('nice', 5569),
 ('long', 4956),
 ('much', 4846),
 ('deep', 4722),
 ('old', 4597),
 ('light', 4255),
 ('first', 4103),
 ('right', 3765),
 ('soft', 3496),
 ('high', 3470),
 ('well', 3380),
 ('black', 3358),
 ('easy', 3238),
 ('large', 3235),
 ('hard', 3207)]

In [ ]:
from spacy.symbols import amod
from pprint import pprint

In [ ]:
def get_amods(noun, ser):
    amod_list = []
    for doc in ser:
        for token in doc:
            if (token.text) == noun:
                for child in token.children:
                    if child.dep == amod:
                        amod_list.append(child.text.lower())
    return sorted(amod_list)

def amods_by_sentiment(noun):
    print(f"Adjectives describing {str.upper(noun)}:\n")
    
    print("POSITIVE:")
    pprint(get_amods(noun, positive_reviews.spacy_doc))
    
    print("\nNEGATIVE:")
    pprint(get_amods(noun, negative_reviews.spacy_doc))